In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import jax
import jax.numpy as jnp
import numpy as np
import flax.linen as nn
import pandas as pd
from functools import partial
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
os.chdir("../../../../")
from utilities.fits import fit
from datasets.dataset_loader import dataset_load
from utilities import plot, gmm, errors, predict, preprocess
from utilities.recalibration_conformal import *
from models import seq2point_gaussian
import time as time
import scipy.stats as st
# from mapie.metrics import regression_coverage_score
from sklearn.isotonic import IsotonicRegression
from tueplots import bundles
import numpyro
import numpyro.distributions as dist
from numpyro.infer import SVI, Trace_ELBO, Predictive
from numpyro.optim import Adam
from numpyro.infer.autoguide import AutoDiagonalNormal
os.environ["XLA_FLAGS"] = "--xla_gpu_deterministic_reductions --xla_gpu_autotune_level=2"
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'


In [ ]:
from jax import random

In [ ]:
# Define the seq2point model using Flax
class Seq2Point(nn.Module):
    @nn.compact
    def __call__(self, X, deterministic):
        X = nn.Conv(30, kernel_size=(10,))(X)
        X = nn.relu(X)
        X = nn.Conv(30, kernel_size=(8,))(X)
        X = nn.relu(X)        
        X = nn.Conv(40, kernel_size=(6,))(X)
        X = nn.relu(X)
        X = nn.Conv(50, kernel_size=(5,))(X)
        X = nn.relu(X)
        X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        X = nn.Conv(50, kernel_size=(5,))(X)
        X = nn.relu(X)
        X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        X = X.reshape((X.shape[0], -1))
        X = nn.Dense(1024)(X)
        X = nn.relu(X)
        X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        mean = nn.Dense(1)(X)
        sigma = nn.softplus(nn.Dense(1)(X))
        return mean, sigma

# Define the model for NumPyro
def model(x, y=None):
    module = Seq2Point()
    params = {
        "params": {
            "Conv_0": {"kernel": numpyro.sample("Conv_0_kernel", dist.Normal(0, 1).expand([10, 1, 30])),
                       "bias": numpyro.sample("Conv_0_bias", dist.Normal(0, 1).expand([30]))},
            "Conv_1": {"kernel": numpyro.sample("Conv_1_kernel", dist.Normal(0, 1).expand([8, 30, 30])),
                       "bias": numpyro.sample("Conv_1_bias", dist.Normal(0, 1).expand([30]))},
            "Conv_2": {"kernel": numpyro.sample("Conv_2_kernel", dist.Normal(0, 1).expand([6, 30, 40])),
                       "bias": numpyro.sample("Conv_2_bias", dist.Normal(0, 1).expand([40]))},
            "Conv_3": {"kernel": numpyro.sample("Conv_3_kernel", dist.Normal(0, 1).expand([5, 40, 50])),
                       "bias": numpyro.sample("Conv_3_bias", dist.Normal(0, 1).expand([50]))},
            "Conv_4": {"kernel": numpyro.sample("Conv_4_kernel", dist.Normal(0, 1).expand([5, 50, 50])),
                       "bias": numpyro.sample("Conv_4_bias", dist.Normal(0, 1).expand([50]))},
            "Dense_0": {"kernel": numpyro.sample("Dense_0_kernel", dist.Normal(0, 1).expand([4950, 1024])),  # Adjusted input size
                        "bias": numpyro.sample("Dense_0_bias", dist.Normal(0, 1).expand([1024]))},
            "Dense_1": {"kernel": numpyro.sample("Dense_1_kernel", dist.Normal(0, 1).expand([1024, 1])),
                        "bias": numpyro.sample("Dense_1_bias", dist.Normal(0, 1).expand([1]))},
            "Dense_2": {"kernel": numpyro.sample("Dense_2_kernel", dist.Normal(0, 1).expand([1024, 1])),
                        "bias": numpyro.sample("Dense_2_bias", dist.Normal(0, 1).expand([1]))}
        }
    }
    rng_key = random.PRNGKey(0)
    mean, sigma = module.apply(params, x, deterministic=True, rngs={"dropout": rng_key})
    mean = numpyro.deterministic("mu", mean)
    sigma = numpyro.deterministic("sigma", nn.softplus(sigma))
    # print(sigma)
    with numpyro.plate("data", x.shape[0]):
        obs = numpyro.sample("obs", dist.Normal(mean, sigma), obs=y)


# SVI Implementation for NILM

In [ ]:
train = {
    2: {
        'start_time': "2013-07-01",
        'end_time': "2013-07-31"
    },
    5: {
        'start_time': "2014-07-01",
        'end_time': "2014-07-31"
    }
}
test = {
    1: {
        'start_time': "2014-07-01",
        'end_time': "2014-07-31"
    }
}
appliances = ["fridge"]

In [ ]:
datas = dataset_load(appliances, train, test, 99, split_factor=0.25)
x_train, y_train = datas[0], datas[1]
x_cal, y_cal = datas[2], datas[3]
x_test, y_test = datas[4], datas[5]
x_test_timestamp = datas[6]
scaler_x, scaler_y = datas[7], datas[8]

In [ ]:
# Define the guide for VI
guide = AutoDiagonalNormal(model)

# Set up the optimizer and inference procedure
optimizer = Adam(step_size=0.1)
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# Initialize inference
rng_key = random.PRNGKey(0)
n_iterations = 500
# init_state = svi.init(rng_key, x_train, y_train)

In [ ]:
svi_result = svi.run(rng_key, n_iterations, x_train, y_train)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
ax.plot(svi_result.losses)
ax.set_title("ELBO loss", fontsize=18, fontweight="bold")

In [ ]:
import arviz as az
import xarray as xr

In [ ]:
obs_test = jnp.arange(x_test.shape[0])

In [ ]:
obs_test.shape

In [ ]:
params = svi_result.params
predictive = Predictive(model=model, guide=guide, params=params, num_samples=5)
rng_key, rng_subkey = random.split(key=rng_key)
test_posterior_predictive_samples = predictive(rng_subkey, x_test)

test_idata_svi = az.from_dict(
    posterior_predictive={
        k: np.expand_dims(a=np.asarray(v), axis=0)
        for k, v in test_posterior_predictive_samples.items()
    },
    coords={"obs": obs_test},
    dims={"mu": ["obs"], "sigma": ["obs"]},
)

test_posterior_predictive_original_scale = {
    var_name: xr.apply_ufunc(
        scaler_y.inverse_transform,
        test_idata_svi["posterior_predictive"][var_name].expand_dims(
            dim={"_": 1}
        ),
        input_core_dims=[["obs", "_"]],
        output_core_dims=[["obs", "_"]],
        vectorize=True,
    ).squeeze(dim="_")
    for var_name in ["mu", "sigma"]
}

In [ ]:
test_mean = test_posterior_predictive_original_scale['mu']
test_sigma = test_posterior_predictive_original_scale['sigma']

In [ ]:
test_mean = test_mean.mean(axis=1)
test_mean = np.array(test_mean)
test_mean = test_mean.reshape(-1,1)
test_sigma = test_sigma.mean(axis=1)
test_sigma = np.array(test_sigma)
test_sigma = test_sigma.reshape(-1,1)

In [ ]:
test_mean = scaler_y.inverse_transform(test_mean)
test_sigma = scaler_y.inverse_transform(test_sigma)

In [ ]:
test_mean

In [ ]:
print(f"RMSE : {errors.rmse(y_test, test_mean):.4f} MAE  : {errors.mae(y_test, test_mean):.4f} NLL : {errors.NLL(test_mean,test_sigma,y_test):.4f}")

# HMC Implementation for NILM

In [ ]:
from numpyro.infer import MCMC, NUTS, Predictive, HMC
import arviz as az
import xarray as xr

In [ ]:
from jax import random

In [ ]:
# Initialize the NUTS sampler
nuts_kernel = NUTS(model, target_accept_prob=0.9)
mcmc = MCMC(nuts_kernel, num_warmup=25, num_samples=400)

# Run inference
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, x_train, y_train)

# Extract the samples
samples = mcmc.get_samples()

In [ ]:
obs_test = jnp.arange(x_test.shape[0])

In [ ]:
x_test.shape

In [ ]:
# obs_test = jnp.arange(x_test[:2000].shape[0])

In [ ]:
device_map='auto'

In [ ]:
# params = svi_result.params
predictive = Predictive(model=model, posterior_samples=samples)
test_posterior_predictive_samples = predictive(jax.random.PRNGKey(0), x_test)

test_idata_svi = az.from_dict(
    posterior_predictive={
        k: np.expand_dims(a=np.asarray(v), axis=0)
        for k, v in test_posterior_predictive_samples.items()
    },
    coords={"obs": obs_test},
    dims={"mu": ["obs"], "sigma": ["obs"]},
)

test_posterior_predictive_original_scale = {
    var_name: xr.apply_ufunc(
        scaler_y.inverse_transform,
        test_idata_svi["posterior_predictive"][var_name].expand_dims(
            dim={"_": 1}
        ),
        input_core_dims=[["obs", "_"]],
        output_core_dims=[["obs", "_"]],
        vectorize=True,
    ).squeeze(dim="_")
    for var_name in ["mu", "sigma"]
}

In [ ]:
# # Making predictions with the posterior samples
# predictive = Predictive(model, num_samples=10)

In [ ]:
# # x_new = jnp.random.randn(30, 1)
# predictions = predictive(random.PRNGKey(1), x_test)

In [ ]:
# # Extract predictions
# predicted_means = jnp.mean(predictions['obs'], axis=0)
# predicted_stddevs = jnp.std(predictions['obs'], axis=0)

# print(f"Predicted mean: {predicted_means}")
# print(f"Predicted stddev: {predicted_stddevs}")

In [ ]:
test_mean = test_posterior_predictive_original_scale['mu']
test_sigma = test_posterior_predictive_original_scale['sigma']

In [ ]:
test_mean = test_mean.mean(axis=1)
test_mean = np.array(test_mean)
test_mean = test_mean.reshape(-1,1)
test_sigma = test_sigma.mean(axis=1)
test_sigma = np.array(test_sigma)
test_sigma = test_sigma.reshape(-1,1)

In [ ]:
# test_mean = scaler_y.inverse_transform(test_mean)
# test_sigma = scaler_y.inverse_transform(test_sigma)
print(f"RMSE : {errors.rmse(y_test, test_mean):.4f} MAE  : {errors.mae(y_test, test_mean):.4f} NLL : {errors.NLL(test_mean,test_sigma,y_test):.4f}")